In [2]:
import logging
from pathlib import Path
import os
import sys
import mne
import torch
import numpy as np
import bm
from bm import play
from bm.train import main
from bm.events import Word
from matplotlib import pyplot as plt
from IPython import display as disp

mne.set_log_level(False)
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
os.chdir(main.dora.dir.parent)
os.environ['NO_DOWNLOAD'] = '1'
from tqdm import tqdm

Hostname megatron not defined in /conf/study_paths/study_paths.yaml. Using default paths.


In [3]:
sig='cf3991ec'

In [4]:
solver = play.get_solver_from_sig(sig) 

INFO:bm.play:Loading solver from XP cf3991ec. Overrides used: ['dset.selections=[audio_mous]', 'num_workers=8', 'optim.batch_size=32', 'hydra.run.dir=/home/druzhinina/brainmagick_MICCAI/outputs/xps/77777755']


INFO:bm.dataset:Loading Subjects | 19/96 | 4.04 it/sec
INFO:bm.dataset:Loading Subjects | 38/96 | 4.34 it/sec
INFO:bm.dataset:Loading Subjects | 57/96 | 4.32 it/sec
INFO:bm.dataset:Loading Subjects | 76/96 | 4.29 it/sec
INFO:bm.dataset:Loading Subjects | 95/96 | 4.36 it/sec
INFO:bm.dataset:# Examples (train | valid | test): 111405 | 17267 | 4684
INFO:bm.train:Model hash: c08252c76253a95a85fc9d0ef9148e3d20db8ef0
ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Currently logged in as: polina. Use `wandb login --relogin` to force relogin


In [5]:
'train'

'train'

In [6]:
solver.datasets.test

In [7]:
len(solver.datasets.test)

4684

In [8]:
len(solver.datasets.test.datasets)

96

In [11]:
solver.datasets.test.datasets[0].recording

Schoffelen2019Recording('sub-A2002')

In [12]:
len(solver.datasets.test.datasets[0])

56

In [13]:
solver.datasets.test.datasets[0].epochs

Number of events,56
Events,1: 56
Time range,-5.000 – 5.000 s
Baseline,-5.000 – 0.000 s


In [14]:
solver.datasets.test.datasets[0].events

,start,duration,kind,context,word,filepath,sequence_id,word_index,phoneme,phoneme_id,word_sequence,sequence_uid,language,modality,offset,uid
0,12.364167,3.840000,sound,sentence,Audio onset,/home/druzhinina/data/meg_data/schoffelen2019/...,186.0,0.0,NaN,NaN,De bands die erg populair zijn brengen enorme ...,186.0,nl,audio,0.0,NaN
1,12.364167,7.933500,block,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nl,audio,NaN,186.0
2,12.404167,0.110000,word,sentence,De,/home/druzhinina/data/meg_data/schoffelen2019/...,186.0,0.0,NaN,NaN,De bands die erg populair zijn brengen enorme ...,186.0,nl,audio,NaN,NaN
3,12.404168,0.059999,phoneme,sentence,De,/home/druzhinina/data/meg_data/schoffelen2019/...,186.0,0.0,d,0.0,De bands die erg populair zijn brengen enorme ...,186.0,nl,audio,NaN,NaN
4,12.464168,0.049999,phoneme,sentence,De,/home/druzhinina/data/meg_data/schoffelen2019/...,186.0,0.0,@,1.0,De bands die erg populair zijn brengen enorme ...,186.0,nl,audio,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13762,2451.268934,0.179999,phoneme,word_list,raad,/home/druzhinina/data/meg_data/schoffelen2019/...,504.0,9.0,t,20.0,was die de ten inbreker pastoor gekrenkte smee...,504.0,nl,audio,NaN,NaN
13763,2451.598933,0.460000,word,word_list,einde,/home/druzhinina/data/meg_data/schoffelen2019/...,504.0,10.0,NaN,NaN,was die de ten inbreker pastoor gekrenkte smee...,504.0,nl,audio,NaN,NaN
13764,2451.598934,0.169999,phoneme,word_list,raad,/home/druzhinina/data/meg_data/schoffelen2019/...,504.0,10.0,Ei,15.0,was die de ten inbreker pastoor gekrenkte smee...,504.0,nl,audio,NaN,NaN
13765,2451.768934,0.119999,phoneme,word_list,raad,/home/druzhinina/data/meg_data/schoffelen2019/...,504.0,10.0,n,4.0,was die de ten inbreker pastoor gekrenkte smee...,504.0,nl,audio,NaN,NaN


In [15]:
solver.datasets.test.datasets[0].features

FeaturesBuilder([('Wav2VecTransformer', Wav2VecTransformer(120.0)),
                 ('WordHash', WordHash(120.0))])

In [16]:
segmentbatch =solver.datasets.test.datasets[0].__getitem__(0)

In [17]:
segmentbatch.meg.shape

torch.Size([273, 1201])

In [18]:
segmentbatch.features.shape

torch.Size([1025, 1201])

In [19]:
def _get_segments_and_vocabs(solver):
    from scripts.run_eval_probs import _get_extra_info
    per_split = {}
    for split in ['test']:
        segments,vocab, estimates,outputs,features_masks, reject_masks = [], [],[], [],[], []
        dset = getattr(solver.datasets, split)
        loader = solver.make_loader(dset, shuffle=False)
        test_features = solver.datasets.test.datasets[0].features
        for idx, batch in tqdm(enumerate(loader)):
            features = test_features.extract_features(batch.features, solver.used_features.keys())
            with torch.no_grad():
                estimate, output, features_mask, reject_mask = solver._process_batch(batch.replace(features=features))
                data,  words, word_segs= _get_extra_info(batch, solver.args.dset.sample_rate)
                segments.append(word_segs)
                vocab.append(words)
                estimates.append(estimate.detach().cpu())
                outputs.append(output.detach().cpu())
        estimates = torch.cat(estimates, dim=0)
        outputs = torch.cat(outputs, dim=0)
        per_split[split] = (segments, vocab, estimates,outputs)
    return per_split

In [20]:
per_split =_get_segments_and_vocabs(solver)

133it [01:21,  1.82it/s]

147it [01:29,  1.65it/s]


In [21]:
segments, vocab, estimates,outputs =per_split['test']

In [22]:
segments[0].shape

(32,)

In [23]:
len(segments)

147

In [24]:
segments[0]

array(['steelt groenteboer de fruit bedelaar van lompen in gehulde de',
       'jaren luisteraars boeiende in miljoenen de zeventig programma het trok',
       'luisteraars boeiende in miljoenen de zeventig programma het trok',
       'had dat Joris penseel was stevige van het de weg korte voorkeur',
       'dat Joris penseel was stevige van het de weg korte voorkeur',
       'Joris penseel was stevige van het de weg korte voorkeur',
       'penseel was stevige van het de weg korte voorkeur',
       'spits de gemist trainer de die cruciale had opstelde training een',
       'de gemist trainer de die cruciale had opstelde training een',
       'rotzooi plein duiven centrale waarschijnlijk op het maken grootste de',
       'duiven centrale waarschijnlijk op het maken grootste de',
       'zusjes met liepen school hand iedere dropje in een naar lekker de',
       'zusjes met liepen school hand iedere dropje in een naar lekker de',
       'bellen vrolijke na van een het taxi feest afloop d

In [25]:
new_segments=[]
for segment in segments:
    for elem in segment:
        new_segments.append(elem)

In [26]:
new_vocab =[]
for voc in vocab:
    for elem in voc:
        new_vocab.append(elem[0])

In [27]:
len(new_segments)

4684

In [28]:
len(new_vocab)

4684

In [29]:
len(vocab)

147

In [30]:
class RepresentationSimilarity():
     def __init__(self,solver,segments, vocab, estimates,outputs):
         self.solver =solver
         self.segments =np.array(segments)
         self.vocab =np.array(vocab)
         self.estimates =estimates
         self.outputs =outputs
         self.clip = solver.loss
         
     def get_item(self, ind):
         return self.segments[ind],self.vocab[ind], self.estimates[ind],self.outputs[ind]

     def get_brain_simmilatity(self, ind, topk, without_target=False):
        segment,vocab,estimate,output = self.get_item(ind)
        if without_target:
            estimates = torch.tensor(np.delete(self.estimates.numpy(), ind,0))
            segments = np.delete(np.copy(self.segments), ind,0)
            probas = self.clip.get_probabilities(estimate[None], estimates)[0]
            print(probas.numpy())
            _, bests = probas.topk(topk)
            print(bests.numpy())
            return self.segments[ind], segments[bests.numpy()],bests.numpy()
        else:
            probas = self.clip.get_probabilities(estimate[None], self.estimates)[0]
            print(probas.numpy())
            _, bests = probas.topk(topk)
            print(bests.numpy())
            return self.segments[ind], self.segments[bests.numpy()],bests.numpy()
         
     def get_audio_simmilatity(self, ind, topk):
        segment,vocab,estimate,output = self.get_item(ind)
        probas = self.clip.get_probabilities(output[None], self.outputs)[0]
        _, bests = probas.topk(topk)
        return self.segments[ind], self.segments[bests.numpy()], bests.numpy()
         
     def decode_brain_activity(self, ind, topk):
        segment,vocab,estimate,output = self.get_item(ind)
        probas = self.clip.get_probabilities(estimate[None], self.outputs)[0]
        _, bests = probas.topk(topk)
        return self.segments[ind], self.segments[bests.numpy()], bests.numpy() 

In [31]:
repsim =RepresentationSimilarity(solver,new_segments, new_vocab, estimates,outputs)

In [32]:
repsim.get_brain_simmilatity(0, 10, without_target=True)

[0. 0. 0. ... 0. 0. 0.]
[1412 1286 1159 2775  179 3840 2464 2315 2056 3799]


('steelt groenteboer de fruit bedelaar van lompen in gehulde de',
 array(['steelt groenteboer de fruit bedelaar van lompen in gehulde de',
        'steelt groenteboer de fruit bedelaar van lompen in gehulde de',
        'voor diepe de een maakte buiging koningin nieuwe man de',
        'steelt groenteboer de fruit bedelaar van lompen in gehulde de',
        'zijn de opbiechtte zonden pastoor broer milde mijn huilde die',
        'zijn de opbiechtte zonden pastoor broer milde mijn huilde die',
        'steelt groenteboer de fruit bedelaar van lompen in gehulde de',
        'sprak heeft ik Hans scheve gisteren torens met nog',
        'steelt groenteboer de fruit bedelaar van lompen in gehulde de',
        'steelt groenteboer de fruit bedelaar van lompen in gehulde de'],
       dtype='<U102'),
 array([1412, 1286, 1159, 2775,  179, 3840, 2464, 2315, 2056, 3799]))

In [33]:
repsim.get_brain_simmilatity(500, 10, without_target=True)

[0. 0. 0. ... 0. 0. 0.]
[2164 2748 1440 1134 4486 3273  193 1908 3182 4589]


('stormachtige vanwege sporthal teams meerdere het in',
 array(['een ons avondeten saus gezin pasta meestal als met lekkere',
        'zaal verjaardag aardige onze hun vieren kleine de hotel buren het in',
        'Joris penseel was stevige van het de weg korte voorkeur',
        'Joris penseel was stevige van het de weg korte voorkeur',
        'is Nora mensen dekens gelukkig warme die voor weeft arme',
        'het de voorgenomen Dodewaard destructie de van in kerncentrale besluit',
        'een ons avondeten saus gezin pasta meestal als met lekkere',
        'geserveerd restaurants pikant in vlees Indische pepers groene',
        'een ons avondeten saus gezin pasta meestal als met lekkere',
        'vastgelegd ingang beroving camera verborgen boven door werd brute de een de'],
       dtype='<U102'),
 array([2164, 2748, 1440, 1134, 4486, 3273,  193, 1908, 3182, 4589]))

In [34]:
repsim.get_brain_simmilatity(800, 10)

[0. 0. 0. ... 0. 0. 0.]
[800   4   8   9   7   3   1   5   0   2]


('scholier maandag gelegd de heeft de die treitert punaises meester stoel de alweer',
 array(['scholier maandag gelegd de heeft de die treitert punaises meester stoel de alweer',
        'dat Joris penseel was stevige van het de weg korte voorkeur',
        'de gemist trainer de die cruciale had opstelde training een',
        'rotzooi plein duiven centrale waarschijnlijk op het maken grootste de',
        'spits de gemist trainer de die cruciale had opstelde training een',
        'had dat Joris penseel was stevige van het de weg korte voorkeur',
        'jaren luisteraars boeiende in miljoenen de zeventig programma het trok',
        'Joris penseel was stevige van het de weg korte voorkeur',
        'steelt groenteboer de fruit bedelaar van lompen in gehulde de',
        'luisteraars boeiende in miljoenen de zeventig programma het trok'],
       dtype='<U102'),
 array([800,   4,   8,   9,   7,   3,   1,   5,   0,   2]))

In [35]:
import random

In [36]:
K=30

In [37]:
for i in range(K):
    indx =random.randint(0,69972)
    segment, similar_segments, seg_ids =repsim.get_brain_simmilatity(indx, 10, without_target=True)
    print(segment)
    print("More similar brain parts:")
    print(similar_segments)
    print("More similar audio fragments")
    segment_audio, similar_segments_audio,seg_au_ids =repsim.get_audio_simmilatity(indx,10)
    print(similar_segments_audio)
    print("Predicted values using brain activity: ")
    segment_brain, decoded_segments, secoded_ids_best =repsim.decode_brain_activity(indx, 10)
    print(decoded_segments)
    print("_____________")

IndexError: index 20306 is out of bounds for axis 0 with size 4684

In [ ]:
segment_brain

'the surge of annoyance that ran through her as she contemplated'

In [ ]:
secoded_ids_best

array([52842, 31422, 24282,  8574, 18570, 45702, 49986, 44274, 22854,
       21426, 15714, 34278, 32850, 11430, 25710, 10002, 37134, 39990,
       14286, 12858])

In [ ]:
decoded_segments

array(['the surge of annoyance that ran through her as she contemplated',
       'the surge of annoyance that ran through her as she contemplated',
       'the surge of annoyance that ran through her as she contemplated',
       'the surge of annoyance that ran through her as she contemplated',
       'the surge of annoyance that ran through her as she contemplated',
       'the surge of annoyance that ran through her as she contemplated',
       'the surge of annoyance that ran through her as she contemplated',
       'the surge of annoyance that ran through her as she contemplated',
       'the surge of annoyance that ran through her as she contemplated',
       'the surge of annoyance that ran through her as she contemplated',
       'the surge of annoyance that ran through her as she contemplated',
       'the surge of annoyance that ran through her as she contemplated',
       'the surge of annoyance that ran through her as she contemplated',
       'the surge of annoyance that ra

In [ ]:
train_seq =per_split['train'][0]

KeyError: 'train'

In [16]:
test_seq =per_split['test'][0]

In [13]:
train_set =set()
for elem in train_seq:
    train_set.update(list(elem))

In [17]:
test_set =set()
for elem in test_seq:
    test_set.update(list(elem))

In [15]:
len(train_set)

7852

In [18]:
len(test_set)

1197

In [20]:
len(set(train_set) & set(test_set))

3

In [23]:
for elem in test_set:
    if 'Harmon ' in elem:
        print(elem)

Harmon she suppressed the surge of annoyance that ran through
Harmon she suppressed the surge of annoyance that ran
Results Harmon she suppressed the surge of annoyance


In [24]:
for elem in train_set:
    if 'Harmon ' in elem:
        print(elem)

Harmon that good nothing son of beta get his
Harmon that good nothing son of beta get


## Посмотрим на распределение аудио и мозговой активности

In [41]:
from torch.utils.data import DataLoader
from bm.dataset import SegmentBatch
from scripts.run_eval_probs import _get_extra_info

In [42]:
defaults = {
            'batch_size': 256,
            'num_workers': 4,
            'collate_fn': SegmentBatch.collate_fn,
            'shuffle': False
        }

In [43]:
dataloader =DataLoader(solver.datasets.test, **defaults)

In [44]:
batch =next(iter(dataloader))

In [45]:
 data1,  words1, word_segs1= _get_extra_info(batch, solver.args.dset.sample_rate)

In [46]:
word_segs1

array(['Results Harmon she suppressed the surge of annoyance',
       'Harmon she suppressed the surge of annoyance that ran',
       'she suppressed the surge of annoyance that ran through',
       'suppressed the surge of annoyance that ran through her as she',
       'suppressed the surge of annoyance that ran through her as she contemplated',
       'suppressed the surge of annoyance that ran through her as she contemplated',
       'the surge of annoyance that ran through her as she contemplated',
       'annoyance that ran through her as she contemplated the',
       'annoyance that ran through her as she contemplated the',
       'that ran through her as she contemplated the gift',
       'ran through her as she contemplated the gift',
       'ran through her as she contemplated the gift of',
       'through her as she contemplated the gift of getting',
       'her as she contemplated the gift of getting',
       'contemplated the gift of getting all the hot news first',
       

In [47]:
defaults = {
            'batch_size': 256,
            'num_workers': 4,
            'collate_fn': SegmentBatch.collate_fn,
            'shuffle': True
        }

In [48]:
dataloader =DataLoader(solver.datasets.test, **defaults)

In [49]:
batch =next(iter(dataloader))

In [50]:
batch

SegmentBatch(meg=tensor([[[-1.3454e-14, -4.0237e-14, -6.1659e-14,  ..., -1.6657e-13,
          -2.3771e-13, -1.6154e-13],
         [-4.0287e-14, -2.4706e-14, -8.6957e-15,  ...,  3.2960e-13,
           3.2682e-13,  3.9180e-13],
         [-7.7937e-14, -6.2094e-14, -2.1099e-13,  ...,  3.9203e-14,
           6.1918e-14,  2.6997e-14],
         ...,
         [ 2.0480e-13,  1.9321e-13,  1.4768e-13,  ..., -4.9384e-13,
          -4.0298e-13, -3.7827e-13],
         [-8.7258e-14, -1.1450e-13, -6.2663e-14,  ...,  1.8316e-13,
           2.0836e-13,  8.9438e-14],
         [-5.9959e-14,  3.1446e-14,  4.4422e-14,  ...,  2.5197e-13,
           3.0399e-13,  2.0258e-13]],

        [[ 7.5983e-14,  6.9774e-14,  1.0413e-13,  ...,  5.3934e-13,
           2.4375e-13,  4.9310e-15],
         [ 3.2659e-14, -5.9550e-14,  1.2657e-13,  ...,  7.9708e-13,
           6.4038e-13,  3.0066e-13],
         [ 9.6183e-14, -6.6715e-15,  5.1828e-14,  ..., -1.6661e-13,
           1.2017e-13,  4.2300e-13],
         ...,
        

In [51]:
batch._event_lists

[[DataSlice(start=742.425, duration=3.0083333333333258, modality=None, language=None, sample_rate=120.0),
  Sound(start=737.817, duration=9.240000000000009, modality='audio', language='english', filepath='/home/zubrikhina/brainmagick_MICCAI/data/gwilliams2022/download/stimuli/audio/easy_money_4.wav', offset=92.58000000000004),
  Word(start=742.237, duration=0.3799999999999954, modality='audio', language='english', word='stepped', word_index=14, word_sequence='For far I could see by a weak gray light stone stepped up and down making little hills and and platforms'),
  Word(start=742.617, duration=0.1800000000000068, modality='audio', language='english', word='up', word_index=15, word_sequence='For far I could see by a weak gray light stone stepped up and down making little hills and and platforms'),
  Word(start=742.797, duration=0.1299999999999954, modality='audio', language='english', word='and', word_index=16, word_sequence='For far I could see by a weak gray light stone stepped up a

In [52]:
data2,  words2, word_segs2= _get_extra_info(batch, solver.args.dset.sample_rate)

In [56]:
len(word_segs2.tolist())

256

In [57]:
len(set(word_segs2))

228

In [58]:
word_segs2

array(['stepped up and down making little hills and and platforms',
       'let forth a moan a cry of purest agony that escaped',
       'ritual or ceremonial gestures a',
       'ancient stories of mankind the tales basic to',
       'feet I could see same design from little journal and',
       'was settling toward the time that Rayburn',
       'the best story ever written',
       'worried for just a moment when I started waving my hands',
       'smelled like first grade He wished he were there now',
       'see by a weak gray light stone stepped up',
       'in which it was his habit walk in a mood',
       'hours on the watch full lips with frustration',
       'pen cast a forbidding line of shadow across the page',
       'friend a bride to be thus upsetting the apple cart',
       'to the ground and seeping though the fallen leaves and needles to join the water',
       'like sleight of hand', 'jerked meat on the bone was wearing',
       'mouth different new deduction magicia